In [1]:
data_var = '2024-01-01'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
8304,2024-01-01,Eua Nba,17:00,New York Knicks,Minnesota Timberwolves,2.05,1.77,221.5,1.84,1.97,-1.5,2.08,1.75,vkQTlyzJ,0.487805,0.564972,0.543478,0.507614,0.052777,0.8,0.8,NaN,NaN,267.468,36.194783,0.135324,200.236,59.989626,0.299595,208.360,232.810,317.34,269.24,0.0,0.0,0.0,0.0,0.103660,0.048254,0.121851,1.32,0.264,3.977273,0.660029,0.8,0.139971,0.64,0.128,6.015625,0.549495,0.7,0.150505
8305,2024-01-01,Eua Nba,21:30,Toronto Raptors,Cleveland Cavaliers,1.85,1.95,223.5,1.85,1.95,-3.5,2.09,1.74,tt3Q8K2O,0.540541,0.512821,0.540541,0.512821,0.053361,0.4,0.2,NaN,NaN,283.526,137.406307,0.484634,239.904,59.435534,0.247747,210.252,350.696,176.12,289.28,0.0,0.0,0.0,0.0,0.037216,0.037216,0.129236,-3.25,-0.650,-1.307692,0.552661,0.4,-0.152661,-0.47,-0.094,-10.106383,0.461348,0.6,0.138652
8306,2024-01-01,Eua Nba,22:00,Houston Rockets,Detroit Pistons,1.36,3.55,225.5,1.80,2.09,-8.5,2.06,1.86,hhEBDKCJ,0.735294,0.281690,0.555556,0.478469,0.016984,0.8,0.6,NaN,NaN,230.300,60.222829,0.261497,586.358,540.339523,0.921518,196.704,1098.926,229.39,1537.20,0.0,1.0,0.0,0.0,0.630780,0.105430,0.072154,-2.17,-0.434,-0.829493,0.575471,0.7,0.124529,-5.00,-1.000,-2.550000,0.186690,0.0,-0.186690
8307,2024-01-01,Eua Nba,22:00,Milwaukee Bucks,Indiana Pacers,1.42,3.23,257.5,1.88,2.05,-7.5,2.06,1.86,jwXECvSP,0.704225,0.309598,0.531915,0.487805,0.013823,0.8,0.4,NaN,NaN,202.266,35.593429,0.175973,247.596,32.078873,0.129561,165.900,268.110,156.94,264.00,0.0,0.0,0.0,0.0,0.550479,0.061175,0.072154,1.27,0.254,1.653543,0.801526,1.0,0.198474,-0.50,-0.100,-22.300000,0.451327,0.6,0.148673
8308,2024-01-01,Eua Nba,23:00,Denver Nuggets,Charlotte Hornets,1.12,8.20,225.5,1.83,2.04,-15.5,2.09,1.85,dK9s8bzs,0.892857,0.121951,0.546448,0.490196,0.014808,0.6,0.6,NaN,NaN,171.374,22.264287,0.129916,726.286,295.073035,0.406277,171.932,736.708,187.44,1124.55,0.0,1.0,0.0,0.0,1.074317,0.076740,0.086145,-0.80,-0.160,-0.750000,0.714453,0.8,0.085547,-5.00,-1.000,-7.200000,0.215077,0.1,-0.115077
8309,2024-01-01,Eua Nba,23:00,Utah Jazz,Dallas Mavericks,2.46,1.64,244.5,1.86,2.07,2.5,2.09,1.85,4pDk6x5g,0.406504,0.609756,0.537634,0.483092,0.016260,0.6,0.6,NaN,NaN,333.762,104.554914,0.313262,342.324,205.775626,0.601114,301.030,391.380,305.00,316.80,0.0,0.0,0.0,0.0,0.282843,0.075569,0.086145,4.64,0.928,1.573276,0.446057,0.6,0.153943,-0.03,-0.006,-106.666667,0.466845,0.6,0.133155
8310,2024-01-01,Arábia Saudita Premier League,12:00,Al Nasr,Al-Hilal,1.35,3.02,166.5,1.85,1.85,-7.5,2.01,1.72,2iFtC2uk,0.740741,0.331126,0.540541,0.540541,0.071867,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,101.12,110.97,0.0,0.0,0.0,0.0,0.540443,0.000000,0.109952,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8311,2024-01-01,Arábia Saudita Premier League,12:00,Al Safa of Safwa,Al Khaleej,1.65,2.15,159.5,1.85,1.85,-3.5,2.00,1.72,trEpBMfe,0.606061,0.465116,0.540541,0.540541,0.071177,0.0,0.0,NaN,NaN,352.088,293.078666,0.832402,NaN,NaN,NaN,0.000,0.000,255.60,401.40,0.0,0.0,0.0,0.0,0.186081,0.000000,0.106446,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8312,2024-01-01,Arábia Saudita Premier League,12:00,Al Salam,Al-Fateh,3.61,1.26,163.5,1.84,1.84,7.5,2.00,1.72,SbCh90P7,0.277008,0.793651,0.543478,0.543478,0.070659,0.0,0.0,NaN,NaN,NaN,NaN,NaN,417.346,478.615887,1.146808,0.000,0.000,474.75,314.53,1.0,0.0,0.0,0.0,0.682423,0.000000,0.106446,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.0000

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,22:00,Eua Nba,Houston Rockets,Detroit Pistons,225.5,1.80,1.0000,Over
1,22:00,Eua Nba,Milwaukee Bucks,Indiana Pacers,257.5,1.88,1.0000,Over
2,23:00,Eua Nba,Denver Nuggets,Charlotte Hornets,225.5,1.83,0.9999,Over
3,23:00,Eua Nba,Utah Jazz,Dallas Mavericks,244.5,1.86,1.0000,Over
4,23:00,Eua Nba,Phoenix Suns,Portland Trail Blazers,228.5,1.86,1.0000,Over
